In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

#load the dataset
data=pd.read_csv("TSLA.csv")
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,29-06-2010,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,30-06-2010,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,01-07-2010,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,02-07-2010,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,06-07-2010,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...,...
2951,18-03-2022,874.489990,907.849976,867.390015,905.390015,905.390015,33408500
2952,21-03-2022,914.979980,942.849976,907.090027,921.159973,921.159973,27327200
2953,22-03-2022,930.000000,997.859985,921.750000,993.979980,993.979980,35289500
2954,23-03-2022,979.940002,1040.699951,976.400024,999.109985,999.109985,40225400


In [ ]:
data=data[['Date','Close']]

In [ ]:
# scaling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
data.loc[:, 'Close'] = scaler.fit_transform(data[['Close']])

In [ ]:
# create sequence

length=60
x,y=[],[]
for i in range(len(data)-length):
  x.append(data['Close'].iloc[i:i+length])
  y.append(data['Close'].iloc[i+length])

In [ ]:
# convert into tensors
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# train test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

x_train = x_train.view(-1, 60, 1)   # (batch, seq_len, features)
y_train = y_train.view(-1, 1)       # (batch, features)
x_test = x_test.view(-1, 60, 1)
y_test = y_test.view(-1, 1)

In [ ]:
# define the model

class ModelLSTM(nn.Module):

  def __init__(self,input_size,hidden_size,num_layers,output_size):
    super(ModelLSTM,self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.fc=nn.Linear(hidden_size,output_size)
  def forward(self,x):
    h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size)
    c0=torch.zeros(self.num_layers,x.size(0),self.hidden_size)

    out,_=self.lstm(x,(h0,c0))
    out=self.fc(out[:,-1,:])
    return out

In [ ]:
#initialization
model=ModelLSTM(1,64,5,1)
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
# training
epochs=100

for i in range(epochs):
  y_pred=model(x_train)
  loss=criterion(y_pred,y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i%10==0:
    print(f"Epoch: {i}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.04120994731783867
Epoch: 10, Loss: 0.04062539339065552
Epoch: 20, Loss: 0.006061213091015816
Epoch: 30, Loss: 0.005463814362883568
Epoch: 40, Loss: 0.0026965495198965073
Epoch: 50, Loss: 0.001259653246961534
Epoch: 60, Loss: 0.0009573749266564846
Epoch: 70, Loss: 0.0008923985878936946
Epoch: 80, Loss: 0.000786363787483424
Epoch: 90, Loss: 0.0007289053173735738


In [ ]:
# testing

with torch.no_grad():
  y_pred=model(x_test)
  loss=criterion(y_pred,y_test)
  print(f"Test Loss: {loss.item()}")

Test Loss: 0.0011674911947920918
